In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

import numpy as np
import os
import torch
import dgl
import dgl.function as fn
flag = "mesh5"
resolution_flag = "32x64"

In [2]:
from model.GraphCast import *

In [3]:
best_model_path = "checkpoints/WeathBench7066/GraphCast/ts_2_pretrain-2D706N_per_1_step/02_07_20_53_42435-seed_73001/backbone.best.pt"

In [4]:
model1 = GraphCast(img_size=(32,64),in_chans=70,out_chans=70,embed_dim=512,depth=6)
model1.load_state_dict(torch.load(best_model_path)['model'],strict=False)


        This is ===> GraphCast Model <===
        Information: 
            total mesh node: 2562 total unique mesh edge:10230 
            total grid node 2048+2 = 2050 but activated grid  1926 +  2
            from activated grid to mesh, create 4*2562 = 10242 edge
            there are 122 unactivated grid node
            when mapping node to grid, 
            from node to activated grid, there are 10242 
            from node to unactivated grid, there are 976 edge
            thus, totally have 11218 edge. 
            #notice some grid only have 1-2 linked node but some grid may have 30 lined node
        


_IncompatibleKeys(missing_keys=[], unexpected_keys=['layer_grid2mesh.G2M_edge_coef_node_tensor', 'layer_mesh2mesh.0.num_of_linked_nodes', 'layer_mesh2mesh.1.num_of_linked_nodes', 'layer_mesh2mesh.2.num_of_linked_nodes', 'layer_mesh2mesh.3.num_of_linked_nodes', 'layer_mesh2mesh.4.num_of_linked_nodes', 'layer_mesh2mesh.5.num_of_linked_nodes', 'layer_mesh2grid.M2G_edge_coef_grid_tensor'])

## Lets chek GraphCastFast

In [6]:
model2 = GraphCastFast(img_size=(32,64),in_chans=70,out_chans=70,embed_dim=512,depth=6)


        This is ===> GraphCast Model (Fast) <===
        Information: 
            total mesh node: 2562 total unique mesh edge:10230 
            total grid node 2048+2 = 2050 but activated grid  1926 +  2
            from activated grid to mesh, create 4*2562 = 10242 edge
            there are 122 unactivated grid node
            when mapping node to grid, 
            from node to activated grid, there are 10242 
            from node to unactivated grid, there are 976 edge
            thus, totally have 11218 edge. 
            #notice some grid only have 1-2 linked node but some grid may have 30 lined node
        


In [7]:
model2.grid_rect_embedding_layer      = model1.grid_rect_embedding_layer
model2.northsouthembbed.data          = model1.northsouthembbed.data
model2.grid_mesh_bond_embedding.data  = model1.grid_mesh_bond_embedding.data[None]
model2.mesh_node_embedding.data       =      model1.mesh_node_embedding.data[None] 
model2.mesh_mesh_bond_embedding.data  = model1.mesh_mesh_bond_embedding.data[None] 
model2.projection = model1.projection

In [321]:
# model2.layer_grid2mesh.STE2E.data    = model1.layer_grid2mesh.MLP_G2M_GM2E.linear.weight.permute(1,0)
# model2.layer_grid2mesh.activator1[1] = model1.layer_grid2mesh.MLP_G2M_GM2E.norm
# model2.layer_grid2mesh.ET2T.data     = model1.layer_grid2mesh.MLP_G2M_E2M.linear.weight.permute(1,0)
# model2.layer_grid2mesh.activator2[1] = model1.layer_grid2mesh.MLP_G2M_E2M.norm
# model2.layer_grid2mesh.S2S.data      = model1.layer_grid2mesh.MLP_G2M_G2G.linear.weight.permute(1,0)
# model2.layer_grid2mesh.activator3[1] = model1.layer_grid2mesh.MLP_G2M_G2G.norm

# for layer1,layer2 in zip(model1.layer_mesh2mesh,model2.layer_mesh2mesh):
#     layer2.STE2E.data    = layer1.MLP_M2M_N2E.linear.weight.permute(1,0)
#     layer2.activator1[1] = layer1.MLP_M2M_N2E.norm
#     layer2.ET2T.data     = layer1.MLP_M2M_E2N.linear.weight.permute(1,0)
#     layer2.activator2[1] = layer1.MLP_M2M_E2N.norm

# model2.layer_mesh2grid.STE2E.data    = model1.layer_mesh2grid.MLP_M2G_MG2E.linear.weight.permute(1,0)
# model2.layer_mesh2grid.activator1[1] = model1.layer_mesh2grid.MLP_M2G_MG2E.norm
# model2.layer_mesh2grid.ET2T.data     = model1.layer_mesh2grid.MLP_M2G_E2G.linear.weight.permute(1,0)
# model2.layer_mesh2grid.activator2[1] = model1.layer_mesh2grid.MLP_M2G_E2G.norm

In [8]:
(model2.layer_grid2mesh.STE2E_E2E.data,
 model2.layer_grid2mesh.STE2E_S2E.data,
 model2.layer_grid2mesh.STE2E_T2E.data)= torch.split(model1.layer_grid2mesh.MLP_G2M_GM2E.linear.weight.permute(1,0).data,512)
(model2.layer_grid2mesh.ET2T_T2T.data,
 model2.layer_grid2mesh.ET2T_E2T.data)= torch.split(model1.layer_grid2mesh.MLP_G2M_E2M.linear.weight.permute(1,0).data,512)
model2.layer_grid2mesh.S2S.data      = model1.layer_grid2mesh.MLP_G2M_G2G.linear.weight.permute(1,0)
model2.layer_grid2mesh.activator1[1] = model1.layer_grid2mesh.MLP_G2M_GM2E.norm
model2.layer_grid2mesh.activator2[1] = model1.layer_grid2mesh.MLP_G2M_E2M.norm
model2.layer_grid2mesh.activator3[1] = model1.layer_grid2mesh.MLP_G2M_G2G.norm

for layer1,layer2 in zip(model1.layer_mesh2mesh,model2.layer_mesh2mesh):
    (layer2.STE2E_E2E.data,
     layer2.STE2E_S2E.data,
     layer2.STE2E_T2E.data)= torch.split(layer1.MLP_M2M_N2E.linear.weight.permute(1,0).data,512)
    (layer2.ET2T_T2T.data,
     layer2.ET2T_E2T.data)= torch.split(layer1.MLP_M2M_E2N.linear.weight.permute(1,0).data,512)
    layer2.activator1[1] = layer1.MLP_M2M_N2E.norm
    layer2.activator2[1] = layer1.MLP_M2M_E2N.norm

(model2.layer_mesh2grid.STE2E_E2E.data,
 model2.layer_mesh2grid.STE2E_S2E.data,
 model2.layer_mesh2grid.STE2E_T2E.data)= torch.split(model1.layer_mesh2grid.MLP_M2G_MG2E.linear.weight.permute(1,0).data,512)
(model2.layer_mesh2grid.ET2T_T2T.data,
 model2.layer_mesh2grid.ET2T_E2T.data)= torch.split(model1.layer_mesh2grid.MLP_M2G_E2G.linear.weight.permute(1,0).data,512)
model2.layer_mesh2grid.activator1[1] = model1.layer_mesh2grid.MLP_M2G_MG2E.norm
model2.layer_mesh2grid.activator2[1] = model1.layer_mesh2grid.MLP_M2G_E2G.norm

In [9]:
fixed_input = torch.randn(2,70,32,64)

In [294]:
with torch.no_grad():
    B, P , W, H =fixed_input.shape
    feature_along_latlot1     = model1.grid_rect_embedding_layer(fixed_input.permute(0,2,3,1).flatten(1,2))
    grid_rect_embedding1      = feature_along_latlot1[:,model1.G2M_grid2LaLotudePos]
    grid_rect_embedding1      = torch.cat([model1.northsouthembbed.repeat(B,1,1),grid_rect_embedding1],1)
    grid_mesh_bond_embedding1 = model1.grid_mesh_bond_embedding.repeat(B,1,1)
    mesh_node_embedding1      = model1.mesh_node_embedding.repeat(B,1,1)
    mesh_mesh_bond_embedding1 = model1.mesh_mesh_bond_embedding.repeat(B,1,1)
    grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1 = model1.layer_grid2mesh(
                                    grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1)
#     for mesh2mesh in model1.layer_mesh2mesh:
#             mesh_mesh_bond_embedding1, mesh_node_embedding1  = mesh2mesh(mesh_mesh_bond_embedding1, mesh_node_embedding1)
    mesh_mesh_bond_embedding1, mesh_node_embedding1  = model1.layer_mesh2mesh[0](mesh_mesh_bond_embedding1, mesh_node_embedding1)

    grid_mesh_bond_embedding1 = torch.nn.functional.pad(grid_mesh_bond_embedding1,(0,0,0,model1.num_unactivated_edge))
    grid_rect_embedding1      = torch.nn.functional.pad(grid_rect_embedding1,(0,0,0,model1.num_unactivated_grid ))
    grid_rect_embedding1      = model1.layer_mesh2grid(grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1)
    grid_rect_embedding1      = grid_rect_embedding1[:,model1.M2G_LaLotudePos2grid] #(B,64,128,embed_dim)
    out1 = model1.projection(grid_rect_embedding1).permute(0,3,1,2)

with torch.no_grad():
    B, P , W, H =fixed_input.shape
    feature_along_latlot2     = model2.grid_rect_embedding_layer(fixed_input.permute(0,2,3,1).flatten(1,2))
    grid_rect_embedding2      = feature_along_latlot2[:,model2.G2M_grid2LaLotudePos]
    grid_rect_embedding2      = torch.cat([model2.northsouthembbed.repeat(B,1,1),grid_rect_embedding2],1)
    grid_mesh_bond_embedding2 = model2.grid_mesh_bond_embedding
    mesh_node_embedding2      = model2.mesh_node_embedding
    mesh_mesh_bond_embedding2 = model2.mesh_mesh_bond_embedding
    
    grid_mesh_bond_embedding2,grid_rect_embedding2,mesh_node_embedding2 = model2.layer_grid2mesh(
                                    grid_mesh_bond_embedding2,grid_rect_embedding2,mesh_node_embedding2)
#     for mesh2mesh in model2.layer_mesh2mesh:
#         mesh_mesh_bond_embedding2, _, mesh_node_embedding2  = mesh2mesh(mesh_mesh_bond_embedding2, mesh_node_embedding2, mesh_node_embedding2)
    mesh_mesh_bond_embedding2, _, mesh_node_embedding2  = model2.layer_mesh2mesh[0](mesh_mesh_bond_embedding2, mesh_node_embedding2,mesh_node_embedding2)

    grid_mesh_bond_embedding2 = torch.nn.functional.pad(grid_mesh_bond_embedding2,(0,0,0,model2.num_unactivated_edge))
    grid_rect_embedding2      = torch.nn.functional.pad(grid_rect_embedding2,(0,0,0,model2.num_unactivated_grid ))
    grid_mesh_bond_embedding2,mesh_node_embedding2,grid_rect_embedding2      = model2.layer_mesh2grid(grid_mesh_bond_embedding2,mesh_node_embedding2,grid_rect_embedding2)
    grid_rect_embedding2      = grid_rect_embedding2[:,model2.M2G_LaLotudePos2grid] #(B,64,128,embed_dim)
    out2 = model2.projection(grid_rect_embedding2).permute(0,3,1,2)

In [10]:
with torch.no_grad():out1 = model1(fixed_input)
with torch.no_grad():out2 = model2(fixed_input)

In [11]:
print(torch.dist(out1,out2))

tensor(0.0002)


In [12]:
import math
def fastinit1(embed_dim1,embed_dim2):
    # for tensor (a,b) `fan_in` deal with b `fan_out` deal with a
    # a linear weight is (out_dim, in_dim) and use `fan_in`
    # but here we direct do matrix matmul, thus fan_out
    return torch.nn.init.kaiming_uniform_(torch.randn(embed_dim1, embed_dim2), mode='fan_out', a=math.sqrt(5)) 

class Node2Edge2NodeBlock(nn.Module):
    def __init__(self,embed_dim=128, do_source_update = False,**kargs):
        super().__init__()
        initial_weight = fastinit1(3*embed_dim, embed_dim)
        STE2E_S2E,STE2E_T2E,STE2E_E2E = torch.split(initial_weight,embed_dim)
        self.STE2E_S2E = nn.Parameter(STE2E_S2E)
        self.STE2E_T2E = nn.Parameter(STE2E_T2E)
        self.STE2E_E2E = nn.Parameter(STE2E_E2E)
        self.activator1 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
        
        initial_weight = fastinit1(2*embed_dim, embed_dim)
        ET2T_E2T,ET2T_T2T = torch.split(initial_weight,embed_dim)
        self.ET2T_E2T  = nn.Parameter(ET2T_E2T)
        self.ET2T_T2T  = nn.Parameter(ET2T_T2T)
        self.activator2 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
        
        self.S2S = None
        if do_source_update:
            self.S2S     = nn.Parameter(fastinit1(embed_dim, embed_dim))
            self.activator3 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))

class Node2Edge2NodeBlockSingleLevel(Node2Edge2NodeBlock):
    def __init__(self, src_tgt_order, edge_order, bond_coef, 
                 embed_dim=128,do_source_update = False, agg_way='mean',
                 **kargs):
        super().__init__(embed_dim=embed_dim,do_source_update = do_source_update)
        self.src_order = src_tgt_order[:,0]
        self.tgt_order = src_tgt_order[:,1]
        self.edge_order= edge_order
        #self.bond_coef = bond_coef
        #self.register_buffer('G2M_edge_coef_node_tensor', G2M_edge_coef_node_tensor)
        self.agg_way  = agg_way
        self.bond_coef  = bond_coef
        # do not use nn.Parameter(bond_coef,requires_grad=False), since we may freeze or unfreeze model
        # or use regist('bond_coef',bond_coef) but will bug in multiGPU mode
        
        self.src_index_limit  = self.src_order.max() + 1
        self.tgt_index_limit  = self.tgt_order.max() + 1
        self.edge_index_limit = len(self.src_order)
    
    def node2bond(self,bond_embedding, src_embedding, tgt_embedding,activator):
        ## compute delta bond embedding
        delta_bond_embedding = (src_embedding @ self.STE2E_S2E)[:,self.src_order] #(B,L,D)
        delta_bond_embedding = delta_bond_embedding + (tgt_embedding @ self.STE2E_T2E)[:,self.tgt_order] #(B,L,D)
        delta_bond_embedding = delta_bond_embedding + bond_embedding @ self.STE2E_E2E  #(B,1,D)
        delta_bond_embedding = activator(delta_bond_embedding) #(B,L,D)
        return delta_bond_embedding
    
    def bond2node(self,bond_embedding, src_embedding, tgt_embedding,activator):
        device = self.STE2E_S2E.device
        self.bond_coef       = self.bond_coef.to(device)
        bond_reduce = bond_embedding@self.ET2T_E2T
        if self.agg_way == 'mean':
            bond_reduce = (bond_reduce[:,self.edge_order]*self.bond_coef.unsqueeze(-1)).mean(-2)
        elif self.agg_way == 'sum':
            bond_reduce = (bond_reduce[:,self.edge_order]*self.bond_coef.unsqueeze(-1)).sum(-2)
        else:
            raise NotImplementedError
        delta_tgt_embedding = bond_reduce + tgt_embedding@self.ET2T_T2T
        delta_tgt_embedding = activator(delta_tgt_embedding)
        return delta_tgt_embedding
    
    def forward(self, bond_embedding, src_embedding, tgt_embedding):
        ### shape checking
        ### all the necessary rect of grid is recorded in G2M_grid2LaLotudePos
        #### we will plus north south point at the begining torch.cat([north_south_embedding,grid_rect_embedding],1)
        assert len(bond_embedding.shape) == len(src_embedding.shape) == len(tgt_embedding.shape) == 3
        assert self.src_index_limit  ==  src_embedding.shape[1]
        assert self.tgt_index_limit  ==  tgt_embedding.shape[1]
        assert self.edge_index_limit == bond_embedding.shape[1] 
        device = self.STE2E_S2E.device
        self.bond_coef = self.bond_coef.to(device)
        delta_bond_embedding = self.node2bond(bond_embedding, src_embedding, tgt_embedding, self.activator1)
        delta_tgt_embedding  = self.bond2node(delta_bond_embedding, src_embedding, tgt_embedding, self.activator2)
        delta_src_embedding  = self.activator3(src_embedding@self.S2S) if self.S2S is not None else 0
        bond_embedding= bond_embedding  + delta_bond_embedding
        src_embedding = src_embedding   + delta_src_embedding
        tgt_embedding = tgt_embedding   + delta_tgt_embedding
        
        return bond_embedding,src_embedding,tgt_embedding

class Node2Edge2NodeBlockMultiLevel(Node2Edge2NodeBlockSingleLevel):
    def bond2node(self,bond_embedding, src_embedding, tgt_embedding,activator):
        device = self.STE2E_S2E.device
        self.bond_coef       = self.bond_coef.to(device)
        delta_bond_embedding = bond_embedding@self.ET2T_E2T
        delta_bond_embedding = torch.nn.functional.pad(delta_bond_embedding,(0,0,0,1))
        # notice the nearby node of each level either 5 or 6, then we use -1 as the padding number.
        # notice some node has multi-level nearby,
        bond_reduce = torch.zeros_like(tgt_embedding)
        for start_node, end_node_list in self.edge_order:
            bond_reduce[:,start_node] += delta_bond_embedding[:,end_node_list].sum(-2)
        bond_reduce = bond_reduce/self.bond_coef
        delta_tgt_embedding = bond_reduce + tgt_embedding@self.ET2T_T2T
        delta_tgt_embedding = activator(delta_tgt_embedding)
        return delta_tgt_embedding
    
    def forward(self, bond_embedding, src_embedding, tgt_embedding):
        ### shape checking
        ### all the necessary rect of grid is recorded in G2M_grid2LaLotudePos
        #### we will plus north south point at the begining torch.cat([north_south_embedding,grid_rect_embedding],1)
        assert len(bond_embedding.shape) == len(src_embedding.shape) == len(tgt_embedding.shape) == 3
        delta_bond_embedding = self.node2bond(bond_embedding, src_embedding, tgt_embedding, self.activator1)
        delta_tgt_embedding = self.bond2node(delta_bond_embedding, src_embedding, tgt_embedding, self.activator2)
        bond_embedding = bond_embedding + delta_bond_embedding
        tgt_embedding  = tgt_embedding  + delta_tgt_embedding
        return bond_embedding,None,tgt_embedding

class GraphCastFast(MeshCast):
    '''
    ---> 1.5 speed up
    Repreduce of GraphCast in Pytorch.
    GraphCast has three part:
    - Grid to Mesh
    - Mesh to Mesh
    - Mesh to Grid
    -------------------------------------
    the input is a tensor (B, P, W, H), but the internal tensor all with shape (B, L ,P)
    where the L equal the node number or edge number.
    '''
    def __init__(self, img_size=(64,128),  in_chans=70, out_chans=70, depth=6, embed_dim=128, graphflag='mesh5', agg_way='mean', nonlinear='swish', **kargs):
        super().__init__()
        flag = graphflag
        resolution_flag=f'{img_size[0]}x{img_size[1]}'
        ROOTPATH=f"GraphCastStructure/{flag}"
        if not os.path.exists(ROOTPATH):
            self.generate_mesh2mesh_graph_static_file(flag)
        M2M_node2position                 = np.load(os.path.join(ROOTPATH   ,f"M2M_node2position.npy"   ))
        M2M_position2node                 = torch.load(os.path.join(ROOTPATH,f"M2M_position2node.pt"   )                 )
        M2M_node2lined_node               = torch.load(os.path.join(ROOTPATH,f"M2M_node2lined_node.pt"   )               )
        # M2M_edge2id                       = torch.load(os.path.join(ROOTPATH,f"M2M_edge2id.pt"   )                       )

        M2M_edgeid2pair                   = np.load(os.path.join(ROOTPATH   ,f"M2M_edgeid2pair.npy"   ))
        M2M_nearby_edge_per_node_per_level= torch.load(os.path.join(ROOTPATH,f"M2M_nearby_edge_per_node_per_level.pt"   ))
        #M2M_nearby_node_per_node_per_level= torch.load(os.path.join(ROOTPATH,f"M2M_nearby_node_per_node_per_level.pt"   ))

        ROOTPATH=f"GraphCastStructure/{flag}/{resolution_flag}"
        if not os.path.exists(ROOTPATH):self.generate_grid2mesh_graph_static_file(flag,img_size[0])
        G2M_grid2LaLotudePos = np.load(os.path.join(ROOTPATH,f"G2M_grid2LaLotudePos.npy"   )     )
        M2G_LaLotudePos2grid = np.load(os.path.join(ROOTPATH,f"M2G_LaLotudeGrid2rect_tensor.npy"))

        # G2M_rect_of_node_tensor    = np.load(os.path.join(ROOTPATH,f"G2M_rect_of_node_tensor.npy"   ) )
        # G2M_rect_distant_tensor    = np.load(os.path.join(ROOTPATH,f"G2M_rect_distant_tensor.npy"   ) )
        G2M_edge_id_of_node_tensor = np.load(os.path.join(ROOTPATH,f"G2M_edge_id_of_node_tensor.npy") )
        G2M_edge_coef_node_tensor  = np.load(os.path.join(ROOTPATH,f"G2M_edge_coef_node_tensor.npy" ) )
        G2M_edge_id2pair_tensor    = np.load(os.path.join(ROOTPATH,f"G2M_edge_id2pair_tensor.npy"   ) )
        # G2M_edge_pos2_id           = torch.load(os.path.join(ROOTPATH,f"G2M_edge_pos2_id.pt"   ))
        # M2G_node_of_rect_tensor    = np.load(os.path.join(ROOTPATH,f"M2G_node_of_rect_tensor.npy"   ))    
        # M2G_node_distant_tensor    = np.load(os.path.join(ROOTPATH,f"M2G_node_of_rect_tensor.npy"   ))    
        M2G_edge_id_of_grid_tensor= np.load(os.path.join(ROOTPATH,f"M2G_edge_id_of_grid_tensor.npy"   ))  
        M2G_edge_coef_grid_tensor = np.load(os.path.join(ROOTPATH,f"M2G_edge_coef_grid_tensor.npy"   ) )  
        M2G_edge_id2pair_tensor   = np.load(os.path.join(ROOTPATH,f"M2G_edge_id2pair_tensor.npy")      )  
        
        embedding_dim = embed_dim
        self.num_unactivated_grid = num_unactivated_grid = len(M2G_edge_id_of_grid_tensor) - len(G2M_grid2LaLotudePos) - 2
        self.num_unactivated_edge = num_unactivated_edge = len(M2G_edge_id2pair_tensor) - len(G2M_edge_id2pair_tensor)
        print(f'''
        This is ===> GraphCast Model (Fast) <===
        Information: 
            total mesh node:{len(M2M_node2position):5d} total unique mesh edge:{len(M2M_edgeid2pair):5d} 
            total grid node {np.prod(img_size)}+2 = {(len(M2G_edge_id_of_grid_tensor))} but activated grid {len(G2M_grid2LaLotudePos):5d} +  2
            from activated grid to mesh, create 4*{len(M2M_node2position)} = {len(G2M_edge_id2pair_tensor)} edge
            there are {num_unactivated_grid} unactivated grid node
            when mapping node to grid, 
            from node to activated grid, there are {len(G2M_edge_id2pair_tensor)} 
            from node to unactivated grid, there are {num_unactivated_edge} edge
            thus, totally have {len(M2G_edge_id2pair_tensor)} edge. 
            #notice some grid only have 1-2 linked node but some grid may have 30 lined node
        ''')
        G2M_edge_id2pair_tensor   = torch.LongTensor(G2M_edge_id2pair_tensor)
        G2M_edge_coef_node_tensor = torch.Tensor(G2M_edge_coef_node_tensor).softmax(-1)
        

        M2M_edgeid2pair                    = torch.LongTensor(M2M_edgeid2pair)
        M2M_nearby_edge_per_node_per_level = [[torch.LongTensor(start_node),torch.LongTensor(linked_edge_list)] for start_node,linked_edge_list in M2M_nearby_edge_per_node_per_level]
        M2M_num_of_linked_nodes            = torch.FloatTensor([len(M2M_node2lined_node[t]) for t in range(len(M2M_node2lined_node))]).unsqueeze(-1)
        
        M2G_edge_id2pair_tensor  = torch.LongTensor(M2G_edge_id2pair_tensor) # (L,2)
        #notice the M2G_edge_id2pair_tensor record (rect_id , node_id) but in this implement
        # the order should be (src,tgt) which is node_id,rect_id
        # lets convert it . 
        M2G_edge_id2pair_tensor  = torch.stack([M2G_edge_id2pair_tensor[:,1],M2G_edge_id2pair_tensor[:,0]],-1)
        
        M2G_edge_of_rect_tensor  = torch.LongTensor(M2G_edge_id_of_grid_tensor)
        M2G_edge_coef_grid_tensor= torch.Tensor(M2G_edge_coef_grid_tensor).softmax(-1)


        M2G_LaLotudePos2grid = torch.LongTensor(M2G_LaLotudePos2grid)
        G2M_grid2LaLotudePos = torch.LongTensor(G2M_grid2LaLotudePos)
        

        

        self.M2G_LaLotudePos2grid = M2G_LaLotudePos2grid
        self.G2M_grid2LaLotudePos = G2M_grid2LaLotudePos
        
        self.layer_grid2mesh = Node2Edge2NodeBlockSingleLevel(G2M_edge_id2pair_tensor,G2M_edge_id_of_node_tensor,G2M_edge_coef_node_tensor, 
                                    embed_dim=embedding_dim,do_source_update=True,agg_way=agg_way)
        
        self.layer_mesh2mesh = nn.ModuleList()
        for i in range(depth):
            self.layer_mesh2mesh.append(Node2Edge2NodeBlockMultiLevel(M2M_edgeid2pair, M2M_nearby_edge_per_node_per_level, M2M_num_of_linked_nodes, 
                                                  embed_dim=embedding_dim))
        
        self.layer_mesh2grid = Node2Edge2NodeBlockSingleLevel(M2G_edge_id2pair_tensor,M2G_edge_of_rect_tensor,M2G_edge_coef_grid_tensor, 
                                         embed_dim=embedding_dim,agg_way=agg_way)
        
        self.grid_rect_embedding_layer = nn.Linear(in_chans,embedding_dim)
        self.northsouthembbed = nn.Parameter(torch.randn(2,embedding_dim))
        
        self.mesh_node_embedding       = nn.Parameter(torch.randn(1,len(M2M_node2position)      ,embedding_dim))
        self.mesh_mesh_bond_embedding  = nn.Parameter(torch.randn(1,len(M2M_edgeid2pair)        ,embedding_dim))
        self.grid_mesh_bond_embedding  = nn.Parameter(torch.randn(1,len(G2M_edge_id2pair_tensor),embedding_dim))

        self.projection      = nn.Linear(embedding_dim,out_chans)

    def forward(self, _input):
        B, P , W, H =_input.shape
        feature_along_latlot     = self.grid_rect_embedding_layer(_input.permute(0,2,3,1).flatten(1,2))
        grid_rect_embedding      = feature_along_latlot[:,self.G2M_grid2LaLotudePos]
        grid_rect_embedding      = torch.cat([self.northsouthembbed.repeat(B,1,1),grid_rect_embedding],1)
        grid_mesh_bond_embedding = self.grid_mesh_bond_embedding
        mesh_node_embedding      = self.mesh_node_embedding
        mesh_mesh_bond_embedding = self.mesh_mesh_bond_embedding
        
        grid_mesh_bond_embedding,grid_rect_embedding,mesh_node_embedding = self.layer_grid2mesh(
                                        grid_mesh_bond_embedding,grid_rect_embedding,mesh_node_embedding)
        for mesh2mesh in self.layer_mesh2mesh:
            mesh_mesh_bond_embedding, _, mesh_node_embedding  = mesh2mesh(mesh_mesh_bond_embedding, mesh_node_embedding, mesh_node_embedding)
        grid_mesh_bond_embedding = torch.nn.functional.pad(grid_mesh_bond_embedding,(0,0,0,self.num_unactivated_edge))
        grid_rect_embedding      = torch.nn.functional.pad(grid_rect_embedding,(0,0,0,self.num_unactivated_grid ))
        grid_mesh_bond_embedding,mesh_node_embedding,grid_rect_embedding      = self.layer_mesh2grid(grid_mesh_bond_embedding,mesh_node_embedding,grid_rect_embedding)
        grid_rect_embedding      = grid_rect_embedding[:,self.M2G_LaLotudePos2grid] #(B,64,128,embed_dim)
        return self.projection(grid_rect_embedding).permute(0,3,1,2)


In [288]:
# class Node2Edge2NodeBlockSingleLevel(nn.Module):
#     def __init__(self, src_tgt_order, edge_order, bond_coef, 
#                  embed_dim=128,do_source_update = False, agg_way='mean',
#                  **kargs):
#         super().__init__()
#         self.src_order = src_tgt_order[:,0]
#         self.tgt_order = src_tgt_order[:,1]
#         self.edge_order= edge_order
#         #self.bond_coef = bond_coef
#         #self.register_buffer('G2M_edge_coef_node_tensor', G2M_edge_coef_node_tensor)
#         self.agg_way  = agg_way
        
#         self.STE2E   = nn.Parameter(fastinit1(3*embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_S2E = nn.Parameter(fastinit1(3*embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_T2E = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_E2E = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         self.activator1 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
#         self.ET2T  = nn.Parameter(fastinit1(2*embed_dim, embed_dim),requires_grad=True)
#         # self.ET2T_E2T  = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         # self.ET2T_T2T  = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         self.activator2 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
#         self.bond_coef  = bond_coef
#         self.S2S = None
#         if do_source_update:
#             self.S2S     = nn.Parameter(fastinit1(embed_dim, embed_dim))
#             self.activator3 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
#         self.src_index_limit  = self.src_order.max() + 1
#         self.tgt_index_limit  = self.tgt_order.max() + 1
#         self.edge_index_limit = len(self.src_order)
#         self.embed_dim = embed_dim
#     def forward(self, bond_embedding, src_embedding, tgt_embedding):
#         ### shape checking
#         ### all the necessary rect of grid is recorded in G2M_grid2LaLotudePos
#         #### we will plus north south point at the begining torch.cat([north_south_embedding,grid_rect_embedding],1)
#         assert len(bond_embedding.shape) == len(src_embedding.shape) == len(tgt_embedding.shape) == 3
#         assert self.src_index_limit  ==  src_embedding.shape[1]
#         assert self.tgt_index_limit  ==  tgt_embedding.shape[1]
#         assert self.edge_index_limit == bond_embedding.shape[1] 
#         device = self.STE2E.device
        
#         ## compute delta bond embedding
#         self.STE2E_E2E,self.STE2E_S2E,self.STE2E_T2E = torch.split(self.STE2E,self.embed_dim)
#         delta_bond_embedding = src_embedding[:,self.src_order] @ self.STE2E_S2E #(B,L,D)
#         delta_bond_embedding = delta_bond_embedding + tgt_embedding[:,self.tgt_order] @ self.STE2E_T2E #(B,L,D)
#         delta_bond_embedding = delta_bond_embedding + bond_embedding @ self.STE2E_E2E  #(B,1,D)
#         delta_bond_embedding = self.activator1(delta_bond_embedding) #(B,L,D)

#         ## compute delta tgt embedding
#         if self.agg_way == 'mean':
#             bond_reduce = (delta_bond_embedding[:,self.edge_order]*self.bond_coef.unsqueeze(-1).to(device)).mean(-2)
#         elif self.agg_way == 'sum':
#             bond_reduce = (delta_bond_embedding[:,self.edge_order]*self.bond_coef.unsqueeze(-1).to(device)).sum(-2)
#         else:
#             raise NotImplementedError
#         #bond_reduce = torch.einsum('blnd,ln->bld',
#         #                           delta_bond_embedding[:,self.edge_order],
#         #                           self.bond_coef) ### 2.6 ms vs 2.2 ms
#         self.ET2T_T2T, self.ET2T_E2T = torch.split(self.ET2T,self.embed_dim)
#         delta_tgt_embedding = bond_reduce@self.ET2T_E2T
#         delta_tgt_embedding = delta_tgt_embedding + tgt_embedding@self.ET2T_T2T
#         delta_tgt_embedding = self.activator2(delta_tgt_embedding)
        
#         delta_src_embedding = self.activator3(src_embedding@self.S2S) if self.S2S is not None else 0
        
#         bond_embedding= bond_embedding  + delta_bond_embedding
#         src_embedding = src_embedding  + delta_src_embedding
#         tgt_embedding = tgt_embedding  + delta_tgt_embedding
        
#         return bond_embedding,src_embedding,tgt_embedding

# class Node2Edge2NodeBlockMultiLevel(nn.Module):
#     def __init__(self, src_tgt_order, edge_order, num_of_linked_nodes, 
#                  embed_dim=128,do_source_update = False,
#                  **kargs):
#         super().__init__()
#         self.src_order            = src_tgt_order[:,0]
#         self.tgt_order            = src_tgt_order[:,1]
#         self.edge_order           = edge_order
        
        
        
#         self.STE2E   = nn.Parameter(fastinit1(3*embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_S2E = nn.Parameter(fastinit1(3*embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_T2E = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         # self.STE2E_E2E = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         self.activator1 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))

#         self.ET2T  = nn.Parameter(fastinit1(2*embed_dim, embed_dim),requires_grad=True)
#         # self.ET2T_E2T  = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         # self.ET2T_T2T  = nn.Parameter(fastinit1(embed_dim, embed_dim),requires_grad=True)
#         self.activator2 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
#         self.num_of_linked_nodes  = num_of_linked_nodes
        
#         self.S2S = None
#         if do_source_update:
#             self.S2S   = nn.Parameter(fastinit1(embed_dim, embed_dim))
#             self.activator3 = nn.Sequential(torch.nn.SiLU(),torch.nn.LayerNorm(embed_dim))
#         self.embed_dim = embed_dim
#     def forward(self, bond_embedding, src_embedding, tgt_embedding):
#         ### shape checking
#         ### all the necessary rect of grid is recorded in G2M_grid2LaLotudePos
#         #### we will plus north south point at the begining torch.cat([north_south_embedding,grid_rect_embedding],1)
#         assert len(bond_embedding.shape) == len(src_embedding.shape) == len(tgt_embedding.shape) == 3
#         device = self.STE2E.device
        
        
#         ## compute delta bond embedding
#         self.STE2E_E2E,self.STE2E_S2E,self.STE2E_T2E = torch.split(self.STE2E,self.embed_dim)
#         delta_bond_embedding = src_embedding[:,self.src_order] @ self.STE2E_S2E #(B,L,D)
#         delta_bond_embedding = delta_bond_embedding + tgt_embedding[:,self.tgt_order] @ self.STE2E_T2E #(B,L,D)
#         delta_bond_embedding = delta_bond_embedding + bond_embedding @ self.STE2E_E2E  #(B,1,D)
#         delta_bond_embedding = self.activator1(delta_bond_embedding) #(B,L,D)

        
#         ## compute delta tgt embedding
#         delta_bond_embedding= torch.nn.functional.pad(delta_bond_embedding,(0,0,0,1))
#         # notice the nearby node of each level either 5 or 6, then we use -1 as the padding number.
#         # notice some node has multi-level nearby,
#         bond_reduce = torch.zeros_like(tgt_embedding)
#         for start_node, end_node_list in self.edge_order:
#             bond_reduce[:,start_node] += delta_bond_embedding[:,end_node_list].sum(-2)
#         bond_reduce = bond_reduce/self.num_of_linked_nodes.to(device) #(B,L,D)/(L)

#         self.ET2T_T2T, self.ET2T_E2T = torch.split(self.ET2T,self.embed_dim)
#         delta_tgt_embedding = bond_reduce@self.ET2T_E2T
#         delta_tgt_embedding = delta_tgt_embedding + tgt_embedding@self.ET2T_T2T
#         delta_tgt_embedding = self.activator2(delta_tgt_embedding)
        
#         ## compute delta src embedding
#         delta_src_embedding   = self.activator3(src_embedding@self.S2S) if self.S2S is not None else 0
        
#         bond_embedding = bond_embedding + delta_bond_embedding[:,:-1]
#         tgt_embedding  = tgt_embedding  + delta_tgt_embedding
        
#         return bond_embedding,None,tgt_embedding

## Lets check GraphCastDGL

In [266]:
import dgl
import dgl.function as fn
from einops import rearrange
class Node2Edge2NodeBlockDGL(Node2Edge2NodeBlock):
    def __init__(self, src_flag, edgetype, tgt_flag, embed_dim=128,do_source_update=False):
        super().__init__(embed_dim=embed_dim,do_source_update = do_source_update)
        self.src_flag = src_flag
        self.tgt_flag = tgt_flag
        self.edgetype = edgetype
    def forward(self,g):
        src_flag = self.src_flag 
        tgt_flag = self.tgt_flag 
        edgetype = self.edgetype 
        edgeflag = (src_flag,edgetype,tgt_flag)
        g.nodes[src_flag].data['src']     = g.nodes[src_flag].data['feat'] @ self.STE2E_S2E
        g.nodes[tgt_flag].data['dst']     = g.nodes[tgt_flag].data['feat'] @ self.STE2E_T2E
        g.apply_edges(fn.u_add_v('src', 'dst', 'node_to_edge'),etype=edgeflag)
        g.edges[edgeflag].data['add_feat'] = self.activator1(g.edges[edgeflag].data['node_to_edge'] + g.edges[edgeflag].data['feat'] @ self.STE2E_E2E)
        g.edges[edgeflag].data['feat']  =   g.edges[edgeflag].data['feat'] + g.edges[edgeflag].data['add_feat']
        if 'coef' in g.edges[edgeflag].data:g.edges[edgeflag].data['add_feat']*= g.edges[edgeflag].data['coef']
        reduce_fun = fn.sum if 'coef' in g.edges[edgeflag].data else fn.mean
        g.update_all(fn.copy_e('add_feat','add_feat'),reduce_fun('add_feat', 'add_feat'),etype=edgeflag)
        g.nodes[tgt_flag].data['add_feat'] = self.activator2(g.nodes[tgt_flag].data['add_feat'] @ self.ET2T_E2T + 
                                                             g.nodes[tgt_flag].data['feat'] @ self.ET2T_T2T)
        g.nodes[tgt_flag].data['feat']  = g.nodes[tgt_flag].data['feat'] + g.nodes[tgt_flag].data['add_feat']
        if self.S2S is not None:g.nodes[src_flag].data['feat'] = g.nodes[src_flag].data['feat']+ self.activator3(g.nodes[src_flag].data['feat']@ self.S2S)
        return g

class GraphCastDGL(MeshCast):    
    '''
    ====>  fastest in atom operation test, but slower than GraphCastFast in practice.
    ====>  still faster than normal GraphCast
    Repreduce of GraphCast in Pytorch.
    GraphCast has three part:
    - Grid to Mesh
    - Mesh to Mesh
    - Mesh to Grid
    -------------------------------------
    the input is a tensor (B, P, W, H), but the internal tensor all with shape (B, L ,P)
    where the L equal the node number or edge number.
    '''
    def __init__(self, img_size=(32,64),  in_chans=70, out_chans=70, depth=6, embed_dim=128, graphflag='mesh5', nonlinear='swish', **kargs):
        super().__init__()
        flag = graphflag
        resolution_flag=f'{img_size[0]}x{img_size[1]}'
        ROOTPATH=f"GraphCastStructure/{flag}"

        M2M_edgeid2pair         = torch.LongTensor(np.load(os.path.join(ROOTPATH,f"M2M_edgeid2pair.npy"   )))
        ROOTPATH=f"GraphCastStructure/{flag}/{resolution_flag}"
        G2M_edge_id2pair_tensor = (np.load(os.path.join(ROOTPATH,f"G2M_edge_id2pair_tensor.npy")))
        #M2G_edge_id2pair_tensor = (np.load(os.path.join(ROOTPATH,f"M2G_id2edge_max_rank.npy")))
        M2G_edge_id2pair_tensor = (np.load(os.path.join(ROOTPATH,f"M2G_id2edge_max_rank.npy")))
        self.G2M_grid2LaLotudePos = np.load(os.path.join(ROOTPATH,f"G2M_grid2LaLotudePos.npy"   )     )
        self.M2G_LaLotudePos2grid = np.load(os.path.join(ROOTPATH,f"M2G_LaLotudeGrid2rect_tensor.npy"))

        
        graph_data = {
                    ('mesh', 'M2M', 'mesh')  : (np.concatenate([M2M_edgeid2pair[:,0], M2M_edgeid2pair[:,1]]),
                                                np.concatenate([M2M_edgeid2pair[:,1], M2M_edgeid2pair[:,0]])),
                    ('grid', 'G2M', 'mesh')  : (G2M_edge_id2pair_tensor[:,0],G2M_edge_id2pair_tensor[:,1]),
                    ('mesh', 'M2G', 'grid')  : (M2G_edge_id2pair_tensor[:,1],M2G_edge_id2pair_tensor[:,0]),
                    
                }
        g = dgl.heterograph(graph_data)

        total_mesh_node = g.num_nodes('mesh')
        total_mesh_edge = g.num_edges('M2M')
        total_grid_node = g.num_nodes('grid') - 2
        total_G2M_edges = g.num_edges('G2M')
        total_M2G_edges = g.num_edges('M2G')
        self.activated_gridn  = activated_gridn  = G2M_edge_id2pair_tensor[:,0].max()+1
        self.unactivated_grid = unactivated_grid = g.num_nodes('grid') - activated_gridn
        
        # notice M2G_edge_id2pair_tensor still use (rect_id, node_id)
        # we firstly select those pair
        shared_index_in_M2G = np.where(M2G_edge_id2pair_tensor[:,0]<=G2M_edge_id2pair_tensor[:,0].max())[0]
        shared_pair_u = M2G_edge_id2pair_tensor[shared_index_in_M2G,0]
        shared_pair_v = M2G_edge_id2pair_tensor[shared_index_in_M2G,1]
        ## now we need find the order of all the ('grid', 'G2M', 'mesh') edge 
        ## in ('mesh', 'M2G', 'grid') subgraph
        ### then we should know the location for each pair in G2M edge list
        ##### one thing omit here is that the egde order in M2G is along the M2G_edge_id2pair_tensor index since we use it to create graph
        self.reorder_edge_id_of_M2G_from_G2M = g.edge_ids(shared_pair_u,shared_pair_v, etype=('grid', 'G2M', 'mesh')) 
        self.reorder_edge_id_in_M2G = shared_index_in_M2G
        M2G_edge_from_node_to_activate_grid = len(self.reorder_edge_id_of_M2G_from_G2M)
        self.num_unactivated_edge = num_unactivated_edge = total_M2G_edges - M2G_edge_from_node_to_activate_grid
        print(f'''
        This is ===> GraphCast Model(DGL) <===
        Information: 
            total mesh node:{total_mesh_node:5d} total unique mesh edge:{total_mesh_edge//2:5d}*2={total_mesh_edge:5d} 
            total grid node {total_grid_node}+2 = {total_grid_node+2} but activated grid {activated_gridn} 
            from activated grid to mesh, create 4*{total_mesh_node} - 6 = {total_G2M_edges} edges. (north and south pole repeat 4 times) 
            there are {unactivated_grid} unactivated grid node
            when mapping node to grid, 
            from node to activated grid, there are {M2G_edge_from_node_to_activate_grid} edges
            from node to unactivated grid, there are {num_unactivated_edge} edges
            thus, totally have {total_M2G_edges} edge. 
            #notice some grid only have 1-2 linked node but some grid may have 30 lined node
        ''')

        G2M_rect_of_node_tensor= np.load(os.path.join(ROOTPATH,f"G2M_rect_of_node_tensor.npy"))  
        G2M_rect_distant_tensor= np.load(os.path.join(ROOTPATH,f"G2M_rect_distant_tensor.npy") )
        G2M_rect_node_tensor= torch.LongTensor(G2M_rect_of_node_tensor)  
        G2M_rect_coef_tensor= torch.softmax(torch.FloatTensor(G2M_rect_distant_tensor),axis=-1)

        #### create_edge_coef_in_grid2mesh ######
        edge_flag = ('grid', 'G2M', 'mesh')
        NRC_tensor = torch.Tensor([(node_id, rect_id, coef) for node_id, (rect_list,coef_list) in enumerate(zip(
            G2M_rect_node_tensor,G2M_rect_coef_tensor)) for rect_id,coef in zip(rect_list,coef_list)])
        edge_idlist = g.edge_ids(NRC_tensor[:,1].long(),NRC_tensor[:,0].long(), etype=edge_flag)    
        edge_ids={}
        for _id,coef in zip(edge_idlist,NRC_tensor[:,2]):
            _id = _id.item()
            if _id not in edge_ids:edge_ids[_id]=0
            edge_ids[_id] += coef
        edge_coef = torch.stack([edge_ids[i] for i in range(len(edge_ids))])
        self.G2M_edge_coef = edge_coef.unsqueeze(-1).unsqueeze(-1)
        #g.edges[edge_flag].data['coef'] = 

        #### create_edge_coef_in_mesh2grid ######
        edge_flag = ('mesh', 'M2G', 'grid')
        M2G_node_of_rect_tensor = np.load(os.path.join(ROOTPATH,f"M2G_node_of_rect_tensor.npy"   ))  
        M2G_node_distant_tensor = np.load(os.path.join(ROOTPATH,f"M2G_node_distant_tensor.npy"   ))  
        M2G_node_distant_tensor[M2G_node_distant_tensor<0]=-100
        M2G_node_of_rect_tensor = torch.LongTensor(M2G_node_of_rect_tensor)  
        M2G_node_distant_tensor = torch.softmax(torch.FloatTensor(M2G_node_distant_tensor),axis=-1)

        NRC_tensor = torch.Tensor([(node_id, rect_id, coef) for rect_id, (node_list,coef_list) in enumerate(zip(
            M2G_node_of_rect_tensor,M2G_node_distant_tensor)) for node_id,coef in zip(node_list,coef_list) if node_id>=0]  )
        edge_idlist = g.edge_ids(NRC_tensor[:,0].long(),NRC_tensor[:,1].long(), etype=edge_flag)  

        edge_ids={}
        for _id,coef in zip(edge_idlist,NRC_tensor[:,2]):
            _id = _id.item()
            if _id not in edge_ids:edge_ids[_id]=0
            edge_ids[_id] += coef
        edge_coef = torch.stack([edge_ids[i] for i in range(len(edge_ids))])
        self.M2G_edge_coef = edge_coef.unsqueeze(-1).unsqueeze(-1)
        #g.edges[edge_flag].data['coef'] = torch.nn.Parameter(edge_coef ,requires_grad=False) # to automatively go into cuda
        edge_flag = ('mesh', 'M2M', 'mesh')
        self.grid2mesh = Node2Edge2NodeBlockDGL('grid','G2M','mesh',embed_dim=embed_dim,do_source_update=True)
        self.mesh2mesh = nn.ModuleList()
        for i in range(depth):
            self.mesh2mesh.append(Node2Edge2NodeBlockDGL('mesh','M2M','mesh',embed_dim=embed_dim))        
        self.mesh2grid = Node2Edge2NodeBlockDGL('mesh','M2G','grid',embed_dim=embed_dim)


        self.grid_rect_embedding_layer = nn.Linear(in_chans,embed_dim)

        self.projection                = nn.Linear(embed_dim,out_chans)

        self.northsouthembbed          = nn.Parameter(torch.randn(2,embed_dim))
        self.mesh_node_embedding       = nn.Parameter(torch.randn(g.num_nodes('mesh')               ,1, embed_dim))
        self.mesh_mesh_bond_embedding  = nn.Parameter(torch.randn(g.num_edges('M2M')                ,1, embed_dim))
        self.grid_mesh_bond_embedding  = nn.Parameter(torch.randn(g.num_edges('G2M'),1, embed_dim))
        self.mesh_grid_bond_template   = torch.randn(g.num_edges('M2G'),1, embed_dim)
        self.g = g
        self.embed_dim = embed_dim
    def forward(self, _input):
        B, P , W, H =_input.shape
        device = next(self.parameters()).device
        self.g = self.g.to(device)
        # (B,P,W,H) -> (B,W*H,P)
        feature_along_latlot     = self.grid_rect_embedding_layer(rearrange(_input,"B P W H -> (W H) B P"))
        grid_rect_embedding      = feature_along_latlot[self.G2M_grid2LaLotudePos] # (L,B,D)
        grid_rect_embedding      = torch.cat([rearrange(self.northsouthembbed.repeat(B,1,1),"B L D -> L B D"),
                                                grid_rect_embedding]) ## --> (L+2, B, D)

        g = self.g
        g.nodes['grid'].data['feat']= torch.nn.functional.pad(grid_rect_embedding,(0,0,0,0,0,self.unactivated_grid))
        g.nodes['mesh'].data['feat']= self.mesh_node_embedding
        g.edges['G2M'].data['feat'] = self.grid_mesh_bond_embedding
        g.edges['M2M'].data['feat'] = self.mesh_mesh_bond_embedding      
        g.edges['G2M'].data['coef'] = self.G2M_edge_coef.to(device)
        g.edges['M2G'].data['coef'] = self.M2G_edge_coef.to(device)
        #checknan(g,'initial');
        g = self.grid2mesh(g)
        #checknan(g,'grid2mesh');
        for layer_idx, mesh2mesh in enumerate(self.mesh2mesh):
            g = mesh2mesh(g);#checknan(g,f"mesh2mesh_{layer_idx}")
        g.edges['M2G'].data['feat'] = torch.nn.functional.pad(g.edges['G2M'].data['feat'][self.reorder_edge_id_of_M2G_from_G2M],(0,0,0,0,0,self.num_unactivated_edge))
        g  = self.mesh2grid(g);#checknan(g,'mesh2grid');
        out = g.nodes['grid'].data['feat'][self.M2G_LaLotudePos2grid] #(64,128,B,embed_dim)
        return self.projection(out).permute(2,3,0,1)

In [267]:
model2 = GraphCastDGL(img_size=(32,64),in_chans=70,out_chans=70,embed_dim=512,depth=6)


        This is ===> GraphCast Model(DGL) <===
        Information: 
            total mesh node: 2562 total unique mesh edge:10230*2=20460 
            total grid node 2048+2 = 2050 but activated grid 1928 
            from activated grid to mesh, create 4*2562 - 6 = 10242 edges. (north and south pole repeat 4 times) 
            there are 122 unactivated grid node
            when mapping node to grid, 
            from node to activated grid, there are 10032 edges
            from node to unactivated grid, there are 976 edges
            thus, totally have 11008 edge. 
            #notice some grid only have 1-2 linked node but some grid may have 30 lined node
        


In [268]:
model2.grid_rect_embedding_layer      = model1.grid_rect_embedding_layer
model2.northsouthembbed.data          = model1.northsouthembbed.data
model2.grid_mesh_bond_embedding.data  = model1.grid_mesh_bond_embedding.data.unsqueeze(1)
model2.mesh_node_embedding.data       =      model1.mesh_node_embedding.data.unsqueeze(1) 
model2.mesh_mesh_bond_embedding.data  = model1.mesh_mesh_bond_embedding.data.unsqueeze(1) 
model2.projection = model1.projection

In [269]:
(model2.grid2mesh.STE2E_E2E.data,
 model2.grid2mesh.STE2E_S2E.data,
 model2.grid2mesh.STE2E_T2E.data)= torch.split(model1.layer_grid2mesh.MLP_G2M_GM2E.linear.weight.permute(1,0).data,512)
(model2.grid2mesh.ET2T_T2T.data,
 model2.grid2mesh.ET2T_E2T.data)= torch.split(model1.layer_grid2mesh.MLP_G2M_E2M.linear.weight.permute(1,0).data,512)
model2.grid2mesh.S2S.data      = model1.layer_grid2mesh.MLP_G2M_G2G.linear.weight.permute(1,0)
model2.grid2mesh.activator1[1] = model1.layer_grid2mesh.MLP_G2M_GM2E.norm
model2.grid2mesh.activator2[1] = model1.layer_grid2mesh.MLP_G2M_E2M.norm
model2.grid2mesh.activator3[1] = model1.layer_grid2mesh.MLP_G2M_G2G.norm

for layer1,layer2 in zip(model1.layer_mesh2mesh,model2.mesh2mesh):
    (layer2.STE2E_E2E.data,
     layer2.STE2E_S2E.data,
     layer2.STE2E_T2E.data)= torch.split(layer1.MLP_M2M_N2E.linear.weight.permute(1,0).data,512)
    (layer2.ET2T_T2T.data,
     layer2.ET2T_E2T.data)= torch.split(layer1.MLP_M2M_E2N.linear.weight.permute(1,0).data,512)
    layer2.activator1[1] = layer1.MLP_M2M_N2E.norm
    layer2.activator2[1] = layer1.MLP_M2M_E2N.norm

(model2.mesh2grid.STE2E_E2E.data,
 model2.mesh2grid.STE2E_S2E.data,
 model2.mesh2grid.STE2E_T2E.data)= torch.split(model1.layer_mesh2grid.MLP_M2G_MG2E.linear.weight.permute(1,0).data,512)
(model2.mesh2grid.ET2T_T2T.data,
 model2.mesh2grid.ET2T_E2T.data)= torch.split(model1.layer_mesh2grid.MLP_M2G_E2G.linear.weight.permute(1,0).data,512)
model2.mesh2grid.activator1[1] = model1.layer_mesh2grid.MLP_M2G_MG2E.norm
model2.mesh2grid.activator2[1] = model1.layer_mesh2grid.MLP_M2G_E2G.norm

In [270]:
fixed_input = torch.randn(2,70,32,64)

In [290]:
with torch.no_grad():
    B, P , W, H =fixed_input.shape
    feature_along_latlot1     = model1.grid_rect_embedding_layer(fixed_input.permute(0,2,3,1).flatten(1,2))
    grid_rect_embedding1      = feature_along_latlot1[:,model1.G2M_grid2LaLotudePos]
    grid_rect_embedding1      = torch.cat([model1.northsouthembbed.repeat(B,1,1),grid_rect_embedding1],1)
    grid_mesh_bond_embedding1 = model1.grid_mesh_bond_embedding.repeat(B,1,1)
    mesh_node_embedding1      = model1.mesh_node_embedding.repeat(B,1,1)
    mesh_mesh_bond_embedding1 = model1.mesh_mesh_bond_embedding.repeat(B,1,1)
    ###########################################################################################
    grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1 = model1.layer_grid2mesh(
                                    grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1,
                                    edge_agg_fun=torch.sum)
    ###   delta_grid_mesh_bond_embedding1 = model1.layer_grid2mesh.MLP_G2M_GM2E(torch.cat([grid_mesh_bond_embedding1,
    ###                               grid_rect_embedding1[:,model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,0]],
    ###                               mesh_node_embedding1[:,model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,1]]],-1))
    ###   delta_mesh_node_embedding1      = model1.layer_grid2mesh.MLP_G2M_E2M(torch.cat([mesh_node_embedding1,
    ###           torch.sum(delta_grid_mesh_bond_embedding1[:,model1.layer_grid2mesh.G2M_edge_id_of_node_tensor]*model1.layer_grid2mesh.G2M_edge_coef_node_tensor.to(device),
    ###                                   -2)],-1))
    ###   delta_grid_rect_embedding1      = model1.layer_grid2mesh.MLP_G2M_G2G(grid_rect_embedding1)
    ###   grid_mesh_bond_embedding1      += delta_grid_mesh_bond_embedding1
    ###   grid_rect_embedding1           += delta_grid_rect_embedding1
    ###   mesh_node_embedding1           += delta_mesh_node_embedding1
    ###########################################################################################
    #mesh_mesh_bond_embedding1, mesh_node_embedding1  = model1.layer_mesh2mesh[0](mesh_mesh_bond_embedding1, mesh_node_embedding1)
    device = model1.layer_mesh2mesh[0].MLP_M2M_N2E.linear.weight.device
    delta_mesh_mesh_bond_embedding1 = model1.layer_mesh2mesh[0].MLP_M2M_N2E(torch.cat([mesh_mesh_bond_embedding1,
                                    mesh_node_embedding1[:,model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,0]],
                                    mesh_node_embedding1[:,model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,1]]],-1))
    mesh_mesh_bond_embedding1 += delta_mesh_mesh_bond_embedding1
    delta_mesh_mesh_bond_embedding1= torch.nn.functional.pad(delta_mesh_mesh_bond_embedding1,(0,0,0,1))
    # notice the nearby node of each level either 5 or 6, then we use -1 as the padding number.
    mesh_node_aggregration = torch.zeros_like(mesh_node_embedding1)
    for start_node, end_node_list in model1.layer_mesh2mesh[0].key_nearbyedge_pair_per_level:
        mesh_node_aggregration[:,start_node] += delta_mesh_mesh_bond_embedding1[:,end_node_list].sum(-2)
    mesh_node_aggregration = mesh_node_aggregration/model1.layer_mesh2mesh[0].num_of_linked_nodes.to(device)

    delta_mesh_node_embedding1 = model1.layer_mesh2mesh[0].MLP_M2M_E2N(torch.cat([mesh_node_embedding1,mesh_node_aggregration],-1))
    mesh_node_embedding1      += delta_mesh_node_embedding1    
    ###########################################################################################
    grid_mesh_bond_embedding1 = torch.nn.functional.pad(grid_mesh_bond_embedding1,(0,0,0,model1.num_unactivated_edge))
    grid_rect_embedding1      = torch.nn.functional.pad(grid_rect_embedding1,(0,0,0,model1.num_unactivated_grid ))
    ###########################################################################################
    #grid_rect_embedding1      = model1.layer_mesh2grid(grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1,
    #                                                       edge_agg_fun=torch.sum)
    mesh_grid_bond_embedding1,grid_allrect_embedding1,mesh_node_embedding1=grid_mesh_bond_embedding1,grid_rect_embedding1,mesh_node_embedding1
    device = model1.layer_mesh2grid.MLP_M2G_MG2E.linear.weight.device
    weight = model1.layer_mesh2grid.MLP_M2G_MG2E.linear.weight
#     delta_mesh_grid_bond_embedding = model1.layer_mesh2grid.MLP_M2G_MG2E(torch.cat([mesh_grid_bond_embedding1,
#                                       mesh_node_embedding1[:,model1.layer_mesh2grid.M2G_edge_id2pair_tensor[:,1]],
#                                    grid_allrect_embedding1[:,model1.layer_mesh2grid.M2G_edge_id2pair_tensor[:,0]]],-1))
    delta_mesh_grid_bond_embedding1 =(torch.cat([
        mesh_grid_bond_embedding1,
        mesh_node_embedding1[:,model1.layer_mesh2grid.M2G_edge_id2pair_tensor[:,1]],
        grid_allrect_embedding1[:,model1.layer_mesh2grid.M2G_edge_id2pair_tensor[:,0]]],-1)@weight.permute(1,0))
    
    delta_grid_rect_embedding1 = torch.sum(delta_mesh_grid_bond_embedding1[:,model1.layer_mesh2grid.M2G_edge_id_of_grid_tensor]*model1.layer_mesh2grid.M2G_edge_coef_grid_tensor.to(device)
                                           ,-2)
    delta_grid_rect_embedding1 =  model1.layer_mesh2grid.MLP_M2G_E2G(
        torch.cat([grid_allrect_embedding1,delta_grid_rect_embedding1
        ],-1))  # notice  <-- this operation should be sum, but we use mean which cause a extra divide
    grid_allrect_embedding1 += delta_grid_rect_embedding1
    #grid_rect_embedding1      = grid_rect_embedding1[:,model1.M2G_LaLotudePos2grid] #(B,64,128,embed_dim)
    #out1 = model1.projection(grid_rect_embedding1).permute(0,3,1,2)

In [15]:
# with torch.no_grad():
#     layer = model1.layer_grid2mesh.MLP_G2M_E2M
#     weight= layer.linear.weight.permute(1,0)
#     normer= layer.norm
#     activator = layer.activator
#     #STE2E_E2E,STE2E_S2E,STE2E_T2E = torch.split(weight,512)
# #     edge_contribution1 = 
# #     node_agg = (edge_contribution+grid_rect_embedding1[:,model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,0]] @STE2E_S2E +
# #                 mesh_node_embedding1[:,model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,1]] @STE2E_T2E)
# #     delta_grid_mesh_bond_embedding0 = normer(activator(node_agg))
#     ET2T_T2T, ET2T_E2T = torch.split(weight,512)
#     edge_contribution1 = torch.mean(delta_grid_mesh_bond_embedding1[:,model1.layer_grid2mesh.G2M_edge_id_of_node_tensor],-2)
#     edge_agg = ( mesh_node_embedding1@ET2T_T2T + edge_contribution1 @ET2T_E2T)
#     delta_mesh_node_embedding0= normer(activator(edge_agg))

In [272]:
g=model2.g

In [273]:
device = next(model2.parameters()).device
model2.g = model2.g.to(device)
# (B,P,W,H) -> (B,W*H,P)

G2Mweightorder= g.edge_ids(model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,0],model1.layer_grid2mesh.G2M_edge_id2pair_tensor[:,1],etype='G2M')

grid_mesh_bond_embedding = torch.zeros_like(model2.grid_mesh_bond_embedding)
grid_mesh_bond_embedding[G2Mweightorder] = model1.grid_mesh_bond_embedding.data.unsqueeze(1)

M2Mweightorder1= g.edge_ids(model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,0],model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,1],etype='M2M')
M2Mweightorder2= g.edge_ids(model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,1],model1.layer_mesh2mesh[0].M2M_edgeid2pair[:,0],etype='M2M')

L,B,D = model2.mesh_mesh_bond_embedding.shape
mesh_mesh_bond_embedding = torch.zeros(2*L,B,D)
mesh_mesh_bond_embedding[M2Mweightorder1] = model1.mesh_mesh_bond_embedding.data.unsqueeze(1)    
mesh_mesh_bond_embedding[M2Mweightorder2] = model1.mesh_mesh_bond_embedding.data.unsqueeze(1)    

In [293]:
with torch.no_grad():
    B, P , W, H =fixed_input.shape
    feature_along_latlot     = model2.grid_rect_embedding_layer(rearrange(fixed_input,"B P W H -> (W H) B P"))
    grid_rect_embedding2      = feature_along_latlot[model2.G2M_grid2LaLotudePos] # (L,B,D)
    grid_rect_embedding2      = torch.cat([rearrange(model2.northsouthembbed.repeat(B,1,1),"B L D -> L B D"),
                                            grid_rect_embedding2]) ## --> (L+2, B, D)

    g.nodes['grid'].data['feat']= torch.nn.functional.pad(grid_rect_embedding2,(0,0,0,0,0,model2.unactivated_grid))
    g.nodes['mesh'].data['feat']= model2.mesh_node_embedding
    g.edges['G2M'].data['feat'] = grid_mesh_bond_embedding
    g.edges['M2M'].data['feat'] = mesh_mesh_bond_embedding
    g.edges['G2M'].data['coef'] = model2.G2M_edge_coef
    g.edges['M2G'].data['coef'] = model2.M2G_edge_coef
    ########################################################################################################
    g = model2.grid2mesh(g)
    ####    src_flag = model2.grid2mesh.src_flag 
    ####    tgt_flag = model2.grid2mesh.tgt_flag 
    ####    edgetype = model2.grid2mesh.edgetype 
    ####    edgeflag = (src_flag,edgetype,tgt_flag)
    ####    g.nodes[src_flag].data['src']     = g.nodes[src_flag].data['feat'] @ model2.grid2mesh.STE2E_S2E
    ####    g.nodes[tgt_flag].data['dst']     = g.nodes[tgt_flag].data['feat'] @ model2.grid2mesh.STE2E_T2E
    ####    g.apply_edges(fn.u_add_v('src', 'dst', 'node_to_edge'),etype=edgeflag)
    ####    agg = g.edges[edgeflag].data['node_to_edge'] + g.edges[edgeflag].data['feat'] @ model2.grid2mesh.STE2E_E2E
    ####    g.edges[edgeflag].data['add_feat'] = model2.grid2mesh.activator1(g.edges[edgeflag].data['node_to_edge'] + g.edges[edgeflag].data['feat'] @ model2.grid2mesh.STE2E_E2E)    
    ####    g.edges[edgeflag].data['feat']  =   g.edges[edgeflag].data['feat'] + g.edges[edgeflag].data['add_feat']
    ####    if 'coef' in g.edges[edgeflag].data:g.edges[edgeflag].data['add_feat']*= g.edges[edgeflag].data['coef']
    ####    reduce_fun = fn.sum
    ####    g.update_all(fn.copy_e('add_feat','add_feat'),reduce_fun('add_feat', 'add_feat'),etype=edgeflag)
    ####    g.nodes[tgt_flag].data['add_feat'] = model2.grid2mesh.activator2(g.nodes[tgt_flag].data['add_feat'] @ model2.grid2mesh.ET2T_E2T +
    ####                                                                     g.nodes[tgt_flag].data['feat'] @ model2.grid2mesh.ET2T_T2T)
    ####    g.nodes[tgt_flag].data['feat']  = g.nodes[tgt_flag].data['feat'] + g.nodes[tgt_flag].data['add_feat']
    ####    if model2.grid2mesh.S2S is not None:
    ####        g.nodes[src_flag].data['feat'] = g.nodes[src_flag].data['feat']+ model2.grid2mesh.activator3(g.nodes[src_flag].data['feat']@ model2.grid2mesh.S2S)
    ########################################################################################################
    #g = model2.mesh2mesh[0](g)
    src_flag = model2.mesh2mesh[0].src_flag 
    tgt_flag = model2.mesh2mesh[0].tgt_flag 
    edgetype = model2.mesh2mesh[0].edgetype 
    edgeflag = (src_flag,edgetype,tgt_flag)
    g.nodes[src_flag].data['src']         = g.nodes[src_flag].data['feat'] @ model2.mesh2mesh[0].STE2E_S2E
    g.nodes[tgt_flag].data['dst']         = g.nodes[tgt_flag].data['feat'] @ model2.mesh2mesh[0].STE2E_T2E
    g.edges[edgeflag].data['edge_effect'] = g.edges[edgeflag].data['feat'] @ model2.mesh2mesh[0].STE2E_E2E
    g.apply_edges(fn.u_add_v('src', 'dst', 'node_effect'),fn.u_add_e('dst', 'add_feat'),etype=edgeflag)
    g.edges[edgeflag].data['add_feat'] = model2.mesh2mesh[0].activator1(g.edges[edgeflag].data['node_to_edge'] + 
                                                                        )
    

In [291]:
    g.edges[edgeflag].data['add_feat'][M2Mweightorder2] = g.edges[edgeflag].data['add_feat'][M2Mweightorder1]

    g.edges[edgeflag].data['feat']    =   g.edges[edgeflag].data['feat'] + g.edges[edgeflag].data['add_feat']
    if 'coef' in g.edges[edgeflag].data:g.edges[edgeflag].data['add_feat']*= g.edges[edgeflag].data['coef']
    reduce_fun = fn.sum if 'coef' in g.edges[edgeflag].data else fn.mean
    g.update_all(fn.copy_e('add_feat','add_feat'),reduce_fun('add_feat', 'add_feat'),etype=edgeflag)
    g.nodes[tgt_flag].data['add_feat'] = model2.mesh2mesh[0].activator2(g.nodes[tgt_flag].data['add_feat'] @ model2.mesh2mesh[0].ET2T_E2T + 
                                                            g.nodes[tgt_flag].data['feat'] @ model2.mesh2mesh[0].ET2T_T2T)
    g.nodes[tgt_flag].data['feat']  = g.nodes[tgt_flag].data['feat'] + g.nodes[tgt_flag].data['add_feat']
    if model2.mesh2mesh[0].S2S is not None:g.nodes[src_flag].data['feat'] = g.nodes[src_flag].data['feat']+ model2.mesh2mesh[0].activator3(g.nodes[src_flag].data['feat']@ model2.mesh2mesh[0].S2S)
    ######################################################################################################
    if model2.mesh_grid_bond_template is None or model2.mesh_grid_bond_template.shape[1]!=B:
        model2.mesh_grid_bond_template = torch.zeros(g.num_edges('M2G'),B,model2.embed_dim).to(g.edges['G2M'].data['feat'].device)
    model2.mesh_grid_bond_template[model2.reorder_edge_id_in_M2G] = g.edges['G2M'].data['feat'][model2.reorder_edge_id_of_M2G_from_G2M]
    g.edges['M2G'].data['feat'] = model2.mesh_grid_bond_template
    g.nodes['grid'].data['feat'][1928:] = 0
#     #######################################################################################################
#     #g  = model2.mesh2grid(g)
    src_flag = model2.mesh2grid.src_flag 
    tgt_flag = model2.mesh2grid.tgt_flag 
    edgetype = model2.mesh2grid.edgetype 
    edgeflag = (src_flag,edgetype,tgt_flag)
    g.nodes[src_flag].data['src']     = g.nodes[src_flag].data['feat'] @ model2.mesh2grid.STE2E_S2E
    g.nodes[tgt_flag].data['dst']     = g.nodes[tgt_flag].data['feat'] @ model2.mesh2grid.STE2E_T2E
    g.apply_edges(fn.u_add_v('src', 'dst', 'node_to_edge'),etype=edgeflag)
    g.edges[edgeflag].data['add_feat'] = (g.edges[edgeflag].data['node_to_edge'] + 
                                          g.edges[edgeflag].data['feat'] @ model2.mesh2grid.STE2E_E2E)
    #g.edges[edgeflag].data['feat']    =   g.edges[edgeflag].data['feat'] + g.edges[edgeflag].data['add_feat']
    if 'coef' in g.edges[edgeflag].data:g.edges[edgeflag].data['add_feat']*= g.edges[edgeflag].data['coef']
    reduce_fun = fn.sum if 'coef' in g.edges[edgeflag].data else fn.mean
    g.update_all(fn.copy_e('add_feat','add_feat'),reduce_fun('add_feat', 'add_feat'),etype=edgeflag)
    g.nodes[tgt_flag].data['add_feat'] = model2.mesh2grid.activator2(g.nodes[tgt_flag].data['add_feat'] @ model2.mesh2grid.ET2T_E2T + 
                                                           g.nodes[tgt_flag].data['feat'] @ model2.mesh2grid.ET2T_T2T)
    g.nodes[tgt_flag].data['feat']  = g.nodes[tgt_flag].data['feat'] + g.nodes[tgt_flag].data['add_feat']
    #if model2.mesh2grid.S2S is not None:g.nodes[src_flag].data['feat'] = g.nodes[src_flag].data['feat']+ model2.mesh2grid.activator3(g.nodes[src_flag].data['feat']@ model2.mesh2grid.S2S)

In [292]:
print(torch.dist(delta_grid_rect_embedding1,g.nodes[tgt_flag].data['add_feat'].permute(1,0,2)))

tensor(0.0007)


In [285]:
M2G_edge_id2pair_tensor_full   = np.load("GraphCastStructure/mesh5/32x64/M2G_edge_id2pair_tensor.npy")    
M2G_edge_id2pair_tensor_sub    = np.load("GraphCastStructure/mesh5/32x64/M2G_id2edge_max_rank.npy") 
M2G_edge_full_edge2id = {}
for u,v in M2G_edge_id2pair_tensor_full:
    M2G_edge_full_edge2id[u,v] = len(M2G_edge_full_edge2id)
subedgeidinfull = [M2G_edge_full_edge2id[u,v] for u,v in M2G_edge_id2pair_tensor_sub]

In [287]:
print(torch.dist(g.nodes['grid'].data['feat'][:1928].permute(1,0,2),grid_rect_embedding1))
print(torch.dist(g.nodes['mesh'].data['feat'].permute(1,0,2),mesh_node_embedding1))
print(torch.dist(g.edges['G2M'].data['feat'].permute(1,0,2),grid_mesh_bond_embedding1))
print(torch.dist(g.edges['M2M'].data['feat'][M2Mweightorder1].permute(1,0,2),mesh_mesh_bond_embedding1))

RuntimeError: The size of tensor a (1928) must match the size of tensor b (2050) at non-singleton dimension 1

In [17]:
((model2.reorder_edge_id_in_M2G[1:]-model2.reorder_edge_id_in_M2G[:1])>0).all()

True

In [223]:
node_effect = torch.sum(delta_mesh_grid_bond_embedding1[:,model1.layer_mesh2grid.M2G_edge_id_of_grid_tensor],-2)

In [282]:
pickout = set(model1.M2G_LaLotudePos2grid.flatten().numpy())

In [254]:
delta = g.nodes[tgt_flag].data['add_feat'] - node_effect.permute(1,0,2)

In [285]:
bad  =set(np.where(np.round(np.linalg.norm(delta.numpy(),axis=(-1,-2)),3)>0.001)[0])

In [289]:
len(bad)

1518

In [287]:
len(pickout&bad)

1516

In [240]:
delta_mesh_grid_bond_embedding1[:,picked_index].sum(1)

tensor([[-11.0468,  -9.5443,   2.4838,  ..., -16.1395, -51.7451,  -7.2725],
        [  5.5867, -13.0053, -11.9248,  ...,  -9.9626, -19.4576,  -7.5640]])

In [216]:
torch.dist(g.edges[edgeflag].data['add_feat'][model2.reorder_edge_id_in_M2G].permute(1,0,2),
           delta_mesh_grid_bond_embedding1[:,model2.reorder_edge_id_of_M2G_from_G2M])

tensor(0.0046)

In [222]:
torch.dist(g.nodes[tgt_flag].data['add_feat'].permute(1,0,2),node_effect)

tensor(3928.8872)

In [198]:
torch.dist(g.edges[edgeflag].data['add_feat'][model2.reorder_edge_id_in_M2G].permute(1,0,2),
           delta_mesh_grid_bond_embedding1[:,model2.reorder_edge_id_of_M2G_from_G2M])

tensor(0.0019)

In [79]:
torch.dist(g.edges['M2M'].data['node_to_edge'][M2Mweightorder1],
           g.edges['M2M'].data['node_to_edge'][M2Mweightorder2])

tensor(3583.1536)

In [65]:
torch.dist(g.edges[edgeflag].data['add_feat'][M2Mweightorder1].permute(1,0,2),delta_mesh_mesh_bond_embedding1)

tensor(0.0012)

In [158]:
print(torch.dist(g.nodes['grid'].data['feat'][:1928].permute(1,0,2),grid_rect_embedding1))
print(torch.dist(g.nodes['mesh'].data['feat'].permute(1,0,2),mesh_node_embedding1))
print(torch.dist(g.edges['G2M'].data['feat'].permute(1,0,2),grid_mesh_bond_embedding1))
print(torch.dist(g.edges['M2M'].data['feat'][M2Mweightorder1].permute(1,0,2),mesh_mesh_bond_embedding1))

tensor(2.0729e-06)
tensor(0.0007)
tensor(0.0014)
tensor(0.0012)


In [77]:

print(torch.dist(g.nodes['mesh'].data['feat'].permute(1,0,2),mesh_node_embedding1))

tensor(2.1806e-06)
tensor(223.1196)
